# SQLAlchemy creating and dropping tables

We've seen how to interact with data in a our database using Pandas but sometimes it is preferable to simply use SQLAlchemy to manipulate our data. Two main cases for this would be table definition and deletion. 

# Create table
We could use an empty Pandas DataFrame with the desired columns and a `to_sql()` call to make an empty table but we can also do this directly with SQLAlchemy. In order to create tables in SQLAlchemy, we need to use the **MetaData** class. We import it as follows:


In [ ]:
from sqlalchemy import MetaData

This **MetaData** object is [" a container object that keeps together many different features of a database (or multiple databases) being described"](https://docs.sqlalchemy.org/en/14/core/metadata.html). We can think of it as holding our table definitions. Speaking of tables, SQLAlchemy also has a **Table** and associated **Column** class for defining a table. Let's import those as well:

In [ ]:
from sqlalchemy import Table, Column


The last set of imports is the the SQLAlchemy data types to use when defining a column in a table. [SQLAlchemy has many supported data types](https://docs.sqlalchemy.org/en/14/core/type_basics.html) but lets import `**String**, **Float**, and **DateTime** to start:

In [ ]:
from sqlalchemy import String, Float, DateTime

And now we have all our imports! To see these in action, let's define a simple table with columns for **name**, **birthday**, and **hight**. Our first step is to instantiate a MetaData object to store all this information:

In [ ]:
meta_store = MetaData()

We then can define out table. The [**Table** constructor](https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.Table) takes the following arguments:
- **name**: the name of the table
- **metadata**: a MetaData object to store the table definition
- any number of comma separated [**Column** objects](https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.Column). These in tern take the following arguments;
  - **name**: name of the column
  - **type**: A SQLAlchemy data type class.
For both **Table** and **Column**, there are additional optional arguments (linked in the above documentation) but these are all we need for now. Let's make the promised table:

In [ ]:
people = Table(
    "people", 
    meta_store,
    Column("name", String(256)), # We need to specify how long our strings will be for SQL
    Column("birthday", DateTime),
    Column("height", Float())
    )

And there's our table definition! However, this just lives locally for now. If we actually want to write it to our database, we need to create an engine to connect to our database as usual:


In [ ]:
from sqlalchemy import create_engine

connection_method = "mysql+pymysql"
db_user = "root"
db_password = "mysql"
db_host = "127.0.0.1"
db_name = "northwind_small"

engine = create_engine(f"{connection_method}://{db_user}:{db_password}@{db_host}/{db_name}", future=True)

To create our table, we can either call the individual table's `create` method with our engine as the argument or if we happened to be defining multiple tables in the same MetaData, we could use the metadata's `create_all` method. Note we can run only one of the following, if we attempt to create a table twice, we get an error the second time since the table already exists. We could use the argument `extends_existing=True` in our Table definition to make this an update instead of a create but for basic creation, this error is the desired outcome if you attempt to create an existing table.

In [ ]:
# Create one table
people.create(engine)

# Create all tables
# meta_store.create_all(engine)

And we can check to see if our empty table was written:

In [ ]:
from sqlalchemy import text
with engine.connect() as conn:
    check = conn.execute(text("SHOW TABLES"))
for row in check:
    print(row)

# Drop table
Similarly, we can drop a table simply by invoking its `drop()` method with our engine:

In [ ]:
people.drop(engine)

from sqlalchemy import text
with engine.connect() as conn:
    check = conn.execute(text("SHOW TABLES"))
for row in check:
    print(row)

We also note that all of the above can be done directly with the SQL query strings we learned how to create in previous episodes. We simply need use `sqlalchemy.text` with the query string as the argument and call our connection's `execute` method to preform the query. We've been doing this for the simple SELECT and SHOW statements but we can do any SQL statement we care to define. Let's take a quick look at what we get back from our `engine` when we execute a `SELECT` query. The object we get back is a [Result](https://docs.sqlalchemy.org/en/14/core/connections.html#sqlalchemy.engine.Result) which usually contains multiple [Rows](https://docs.sqlalchemy.org/en/14/core/connections.html#sqlalchemy.engine.Row). We saw this with our `print` check of our above table check or in the previous notebook where looked at the rows of our `orders` table:

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM orders LIMIT 5"))
    for row in result:
        print(row)

In practice, we will almost always use these `text` queries with the Pandas `read_sql` function as it automatically creates a DataFrame for us from the results instead of giving us the raw Result with Rows. However, we can also work directly with the Rows if we want to.

# Insert
We saw how to define a table using just SQLAlchemy, let's quickly learn how to insert the data into the table. We can use SQLAlchemy's `insert` function for this. To create an insert statement, we simply need to pass an SQLAlchemy Table as the argument. This returns an [Insert object](https://docs.sqlalchemy.org/en/14/core/dml.html#sqlalchemy.sql.expression.Insert). To actually supply the values we are inserting, we can create a list of dicts and pass these to the `values()` function of our `insert` object. These dicts have keys for each column and a corresponding value for each 

In [ ]:
from sqlalchemy import insert
from datetime import datetime

# Create the table if you dropped it:
# people.create(engine)

insert_statement = insert(people).values([
    {"name":"Laura Palmer", "birthday":datetime(year=1971, month=7, day=22), "height":65},
    {"name":"Dale Cooper", "birthday":datetime(year=1954, month=4, day=19), "height":72}
])
print(insert_statement)


We then need to execute this statement using our usual connections. We note that SQLAlchemy does not automatically commit the transaction so we can explicitly call the `connect().commit()` so we can be sure that they are committed for us to check with a quick query:

In [ ]:
with engine.connect() as conn:
    conn.execute(insert_statement)
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM people"))
    for row in result:
        print(row)

We also note that we can set the[ values within an execute statement using the `:parameter` ](https://docs.sqlalchemy.org/en/14/tutorial/dbapi_transactions.html#sending-parameters) notation we see in our print of the insert statement we made above. For example, let's say we want to insert three new people into our table. We could do this with another `insert` statement without the `values` argument or with a `text` statement. Let's go with the second for readability. Here's how we would insert three more people:

In [ ]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO people (name, birthday, height) VALUES (:name, :birthday, :height)"),
        [{"name": "Ed", "birthday": datetime(year=1990, month=1, day=4), "height":62},
        {"name": "Edd", "birthday": datetime(year=1990, month=1, day=2), "height":40},
        {"name": "Eddy", "birthday": datetime(year=1990, month=2, day=5), "height":50}]
    )
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM people"))
    for row in result:
        print(row)

This same pattern can be used with any `text` query we want to pass parameters into and avoid some of the SQL insertion vulnerabilities. This flexibility further shows us how we can use `text` to perform any SQL we know how to write.

# Update
Similarly to INSERTING, we sometimes want to UPDATE our data. We can do this with SQLAlchemy directly using `sqlachemy.update` or with a `text` call. Let's update the `height`s of Ed and Eddy to illustrate both methods. [The `update` function](https://docs.sqlalchemy.org/en/14/core/dml.html#sqlalchemy.sql.expression.update) operates very similar to insert but we need to use the `where()` method to tell it what rows to update. Within the `where()` we access the column from `tablename.c.columnname` and can perform any comparison we would like with this column. The `text` statement is the standard `UPDATE` we learned before but we can use the `:` notation with parameters to set our values:

In [ ]:
from sqlalchemy import update

update_statement = update(people).where(people.c.name == "Ed").values({"height": 65})
print(update_statement)

with engine.connect() as conn:
    conn.execute(update_statement)
    conn.execute(
        text("UPDATE people SET height=:height WHERE people.name = :name"),
        {"name": "Eddy", "height":55}
    )
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM people"))
    for row in result:
        print(row)

# Delete row
Lastly, we can delete specify rows. The `sqlachemy.delete` function takes a `where()` clause to specify what rows will be deleted. We can also use `text` again if we prefer. Let's delete our Ed(dy)s using this functions and the Column function `startswith()`:

In [ ]:
from sqlalchemy import delete

delete_statement = delete(people).where(people.c.name.startswith("Ed"))

with engine.connect() as conn:
    conn.execute(delete_statement)
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM people"))
    for row in result:
        print(row)

There are many more functions built into SQLAlchemy which allow all manner of manipulations of our database and the data it contains. Most will follow the above patterns and as we have noted repeatedly, the `text` option exists so we can always fall back onto the SQL we have learned if we do no know the exact SQL Alchemy functionality.